# 構造の厳密なリスト 

In [1]:
from py_strict_list_ver1 import StructureStrictList, TypeStrictList, LengthStrictList

## 型・長さ構造が厳密なリスト

In [34]:
a = StructureStrictList([1,2],["a", "b"])

Exception: list like object have to have same type items

In [35]:
a = StructureStrictList([1,2],[3])

Exception: list like object have to have same length recursive

In [36]:
a = StructureStrictList([1,2],[3,4])

In [37]:
a.length_structure

{2: {2: None}}

In [38]:
a.type_structure

[[int]]

### 他のSSLとの比較

In [39]:
b = StructureStrictList(3,4)
a.check_same_structure_with(b)

False

### 他のリストとの比較

In [40]:
c = [[5,6],[7,8]]
a.check_same_structure_with(c)

True

### 要素との比較 

appendとかの型判定で利用

In [41]:
a.check_item_structure([1,2])

True

In [42]:
a.check_item_structure([3])

False

### append 

In [43]:
a.append(1)

Exception: this item is restricted for append

In [44]:
a.append([5,6])
a

[[1, 2], [3, 4], [5, 6]]

###  extend

In [45]:
a.extend([[7,8],[9,10],[11,12]])
a

[[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]]

### from_list 

In [46]:
d = StructureStrictList.from_list([1,2,3])
d

[1, 2, 3]

In [47]:
d.length_structure

{3: None}

In [48]:
d.type_structure

[int]

### 構造から空のSSLを作る 

In [49]:
e = StructureStrictList.from_structures([[int]],{2: {2: None}})
e

[]

In [50]:
e.append(1)

Exception: this item is restricted for append

In [51]:
e.append([1,2])
e

[[1, 2]]

### セッターの定義時について 

リストをプロパティとする場合，セッターを

```python
@some_list.setter
def some_list(self, __some_list):
    if not self._some_list.check_same_structure_with(__some_list, include_outer_length=False):
        raise Exception("This some_list is invalid")
    self._some_list = StructureStrictList.from_list(__some_list)
```

とすれば，型を比較しリストを更新できる．

## 型が厳密なリスト 

In [52]:
class TypeStrictList(StructureStrictList):
    def _get_structure(self):
        """
        構造を取得するためのメソッド．基本的に自身が変更されたときに呼ぶ
        """
        self._type_structure = self._get_type_structure(self)
    
    def check_same_structure_with(self, list_like, include_outer_length=True):
        """
        list_like: list like object
            構造を自身と比較したいリスト
        include_outer_length int
            最も外側の長さを比較する構造に含めるかどうか．今回は意味ない
        """
        try:
            list_like_type_structure = self._get_type_structure(list_like)
        except:  #構造の取得に失敗した場合
            return False
        is_same_type_structure = self._check_same_type_structure(self._type_structure, list_like_type_structure)
        return is_same_type_structure
    
    def check_item_structure(self, item):
        """
        item: any
            構造を自身の要素と比較したい要素候補
        """
        try:
            item_type_structure = self._get_type_structure(item)
        except:  #構造の取得に失敗した場合
            return False
        
        # itemがリストの場合
        if isinstance(item, list):
            is_same_type_structure = self._check_same_type_structure(self._type_structure[0], item_type_structure)
        else:
            if not isinstance(self._type_structure[0], list):
                is_same_type_structure = isinstance(item,self._type_structure[0])
            else:
                is_same_type_structure = False
        
        return is_same_type_structure
    
    @classmethod
    def from_type_structure(cls, _type_structure):
        """
        型構造構造から，空のStructureStrictListを作成
        """
        instance = cls(None)
        super(StructureStrictList, instance).remove(None)  # 親のスーパークラスのメソッド呼び出し
        instance._type_structure = _type_structure
        return instance

In [53]:
a = TypeStrictList(["a","b"],[1,2])

Exception: list like object have to have same type items

In [54]:
a = TypeStrictList(["a","b"],"c")

Exception: list like object have to have same type recursive

In [55]:
a = TypeStrictList(["a","b"],["c","d"])

In [56]:
a.type_structure

[[str]]

### append 

In [57]:
a.append("a")

Exception: this item is restricted for append

In [58]:
a.append(["e"])
a

[['a', 'b'], ['c', 'd'], ['e']]

In [59]:
b = TypeStrictList.from_type_structure([str])

In [60]:
b.type_structure

[str]

In [61]:
b.append(["c"])

Exception: this item is restricted for append

In [62]:
b.append("a")
b

['a']

## 長さが厳密なリスト

In [63]:
class LengthStrictList(StructureStrictList):
    def _get_structure(self):
        """
        構造を取得するためのメソッド．基本的に自身が変更されたときに呼ぶ
        """
        self._length_structure = self._get_length_structure(self)
       
    def check_same_structure_with(self, list_like, include_outer_length=True):
        """
        list_like: list like object
            構造を自身と比較したいリスト
        include_outer_length int
            最も外側の長さを比較する構造に含めるかどうか．
        """
        try:
            list_like_length_structure = self._get_length_structure(list_like)
        except:  #構造の取得に失敗した場合
            return False
        if include_outer_length: # 一番外側の比較も行う
            is_same_length_structure = self._check_same_length_structure(self._length_structure,
                                                                         list_like_length_structure
                                                                        ) 

        else:  # 一番外側の比較は行わない
            is_same_length_structure = self._check_same_length_structure(self._length_structure[list(self._length_structure.keys())[0]],
                                                                         list_like_length_structure[list(list_like_length_structure.keys())[0]]
                                                                        )

        return is_same_length_structure
    
    def check_item_structure(self, item):
        """
        item: any
            構造を自身の要素と比較したい要素候補
        """
        try:
            item_length_structure = self._get_length_structure(item)
        except:  #構造の取得に失敗した場合
            return False
        # itemがリストの場合
        if isinstance(item, list):
            is_same_length_structure = self._check_same_length_structure(self._length_structure[list(self._length_structure.keys())[0]],
                                                                         item_length_structure)
        else:
            is_same_length_structure = True
        
        return is_same_length_structure
    
    @classmethod
    def from_length_structure(cls, _length_structure):
        instance = cls(None)
        super(StructureStrictList, instance).remove(None)  # 親のスーパークラスのメソッド呼び出し
        instance._length_structure = _length_structure
        return instance

In [64]:
a = LengthStrictList([1,2,3],[1,2])

Exception: list like object have to have same length recursive

In [65]:
a = LengthStrictList([[1,2,3],[1,2]])

Exception: list like object have to have same length recursive

In [66]:
a = LengthStrictList([2,3],[1,2])
a

[[2, 3], [1, 2]]